In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import gc

dirThis = '/content/drive/MyDrive/1經濟學/專題/'

offers = pd.read_csv(dirThis + 'offers.csv')
#transactions = pd.read_csv(r'/content/drive/MyDrive/1經濟學/專題/newdata.csv')
trainHistory = pd.read_csv(dirThis + 'trainHistory.csv')
testHistory = pd.read_csv(dirThis + 'testHistory.csv')

tranDtype = {
    'id': 'uint64',
    'chain': 'uint16',
    'dept': 'uint8',
    'category': 'uint16',
    'company': 'uint64',
    'brand': 'uint32',
    'date' : 'object',
    'productsize': 'float32',
    'productmeasure': 'category',
    'purchasequantity': 'int64',
    'purchaseamount': 'float32'
    }


def generateFeature(offers, transactions, trainHistory, testHistory, tidx) :
    # import useful package
    from datetime import timedelta
    from itertools import cycle
    import pandas as pd
    import numpy as np
    import time
    import gc
    ts = time.time()
    # tk = 1
    
    # get all data & delete those not in transactions
    train = trainHistory.drop(columns = ['repeater', 'repeattrips'])
    data = pd.concat([train, testHistory], axis=0, ignore_index=True)
    use = data[data['id'].isin(transactions['id'])] # (310665, )

    del trainHistory, train, testHistory
    gc.collect()
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(get all data) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    # put offer information into transactions
    of = offers[['offer', 'category', 'company', 'offervalue', 'brand']]
    usf = pd.merge(use, of, on='offer')
    usf.columns = ['id', 'chain', 'offer', 'market', 'offerdate', 'offercategory', 'offercompany',
          'offervalue', 'offerbrand']
    tu = usf[['id', 'offer', 'offerdate', 'offercategory', 'offercompany', 'offerbrand']]
    nu = pd.merge(tu, transactions, on='id')
    
    del tu, usf, of, transactions
    gc.collect()
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(put offer information) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    # generate time index
    date_format = '%Y-%m-%d'
    nu['offerdate'] = pd.to_datetime(nu['offerdate'], format = date_format)
    # nu['date'] = pd.to_datetime(nu['date'], format = date_format)
    nu['daydiff'] = nu['offerdate'] - nu['date']
    nu['diff_180'] = nu['offerdate'] - timedelta(days = 180)
    nu['diff_150'] = nu['offerdate'] - timedelta(days = 150)
    nu['diff_120'] = nu['offerdate'] - timedelta(days = 120)
    nu['diff_90'] = nu['offerdate'] - timedelta(days = 90)
    nu['diff_60'] = nu['offerdate'] - timedelta(days = 60)
    nu['diff_30'] = nu['offerdate'] - timedelta(days = 30)
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate time index) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    # put offervalue
    of1 = offers[['offer', 'offervalue']]
    use = pd.merge(use, of1, on='offer')
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(put offervalue) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    # generate total
    group = nu.groupby(['id'])
    
    test = group['chain'].count().reset_index()
    test.columns = ['id', 'buy_total_freq']
    use = pd.merge(use, test, on='id')
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total freq) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    test = group['purchaseamount'].sum().reset_index()
    test.columns = ['id', 'buy_total_amount']
    use = pd.merge(use, test, on='id')
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total amount) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    test = group['purchaseamount'].mean().reset_index()
    test.columns = ['id', 'buy_total_avgamount']
    use = pd.merge(use, test, on='id')
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total avg) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    test = group['purchasequantity'].sum().reset_index()
    test.columns = ['id', 'buy_total_quantity']
    use = pd.merge(use, test, on='id')
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total quan) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    test = group['daydiff'].min().reset_index()
    test.columns = ['id', 'buy_total_daydiff']
    use = pd.merge(use, test, on='id')
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    
    day = np.linspace(30, 180, 6, endpoint=True).astype(int).astype(str)
    
    for i in day :
      daa = 'diff_' + i
      nu['ascom'] = nu['date'] >= nu[daa]
    
      name = 'buy_total_amount_' + i
      var = 'purchaseamount'
    
      group = nu.groupby(['id', 'ascom'])
      test = group[var].sum().reset_index()
      test.columns = ['id', 'ascom', name]
      use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate day amount) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      name = 'buy_total_quantity_' + i
      var = 'purchasequantity'
    
      group = nu.groupby(['id', 'ascom'])
      test = group[var].sum().reset_index()
      test.columns = ['id', 'ascom', name]
      use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate day quan) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      name = 'buy_total_freq_' + i
      var = 'purchaseamount'
    
      group = nu.groupby(['id', 'ascom'])
      test = group['chain'].count().reset_index()
      test.columns = ['id', 'ascom', name]
      use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate day) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
    # generate company & brand & category
    mea = ['company', 'brand', 'category']
    
    for i in mea :
      nu['ascom'] = nu[i] == nu['offer' + i]
      group = nu.groupby(['id', 'ascom'])
    
      test = group['chain'].count().reset_index()
      test.columns = ['id', 'ascom', 'buy_'+i+'_freq']
      use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_freq']], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other freq) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      test = group['purchaseamount'].sum().reset_index()
      test.columns = ['id', 'ascom', 'buy_'+i+'_amount']
      use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_amount']], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other amount) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      test = group['purchaseamount'].mean().reset_index()
      test.columns = ['id', 'ascom', 'buy_'+i+'_avgamount']
      use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_avgamount']], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other avg) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      test = group['purchasequantity'].sum().reset_index()
      test.columns = ['id', 'ascom', 'buy_'+i+'_quantity']
      use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_quantity']], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other quan) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      test = group['daydiff'].min().reset_index()
      test.columns = ['id', 'ascom', 'buy_'+i+'_daydiff']
      use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_daydiff']], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
    
    for i, j in zip(day, cycle(mea)) :
      daa = 'diff_' + i
      nu['ascom'] = ((nu['date'] >= nu[daa]) & (nu[j] == nu['offer' + j]))
    
      name = 'buy_'+j+'_amount_' + i
      var = 'purchaseamount'
    
      group = nu.groupby(['id', 'ascom'])
      test = group[var].sum().reset_index()
      test.columns = ['id', 'ascom', name]
      use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other day amount) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      name = 'buy_'+j+'_quantity_' + i
      var = 'purchasequantity'
    
      group = nu.groupby(['id', 'ascom'])
      test = group[var].sum().reset_index()
      test.columns = ['id', 'ascom', name]
      use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other day quan) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      name = 'buy_'+j+'_freq_' + i
      var = 'purchaseamount'
    
      group = nu.groupby(['id', 'ascom'])
      test = group['chain'].count().reset_index()
      test.columns = ['id', 'ascom', name]
      use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    
      # te = time.time()
      # # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other day) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # # tk+=1
    
    # generate not buy index
    nu['ascom'] = (nu['company'] == nu['offercompany']) & (nu['brand'] == nu['offerbrand']) & (nu['category'] == nu['offercategory'])
    group = nu.groupby(['id', 'ascom'])
    
    name1 = 'buy_company_brand_category'
    name = name1 + '_freq'
    
    test = group['chain'].count().reset_index()
    test.columns = ['id', 'ascom', name]
    use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    use['not_' + name1] = (use[name] > 0) != True
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate not buy) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    nu['ascom'] = (nu['company'] == nu['offercompany']) & (nu['brand'] == nu['offerbrand'])
    group = nu.groupby(['id', 'ascom'])
    
    name1 = 'buy_company_brand'
    name = name1 + '_freq'
    
    test = group['chain'].count().reset_index()
    test.columns = ['id', 'ascom', name]
    use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    use['not_' + name1] = (use[name] > 0) != True
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate not buy) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    nu['ascom'] = (nu['company'] == nu['offercompany']) & (nu['category'] == nu['offercategory'])
    group = nu.groupby(['id', 'ascom'])
    
    name1 = 'buy_company_category'
    name = name1 + '_freq'
    
    test = group['chain'].count().reset_index()
    test.columns = ['id', 'ascom', name]
    use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    use['not_' + name1] = (use[name] > 0) != True
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate not buy) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    nu['ascom'] = (nu['brand'] == nu['offerbrand']) & (nu['category'] == nu['offercategory'])
    group = nu.groupby(['id', 'ascom'])
    
    name1 = 'buy_brand_category'
    name = name1 + '_freq'
    
    test = group['chain'].count().reset_index()
    test.columns = ['id', 'ascom', name]
    use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    use['not_' + name1] = (use[name] > 0) != True

    del test, group
    gc.collect()
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate not buy) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    use['not_buy_company'] = (use['buy_company_freq'] > 0) != True
    use['not_buy_brand'] = (use['buy_brand_freq'] > 0) != True
    use['not_buy_category'] = (use['buy_category_freq'] > 0) != True

    del nu
    gc.collect()
    
    # handle na problem
    dayVar = ['buy_company_daydiff', 'buy_brand_daydiff', 'buy_category_daydiff', 'buy_total_daydiff']
    use1 = use[dayVar]
    use.drop(columns = dayVar, inplace = True)
    use = use.fillna(0)
    use1 = use1.fillna(timedelta(0))
    use[dayVar] = use1

    del use1
    gc.collect()
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(handle na problem) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    # transform date type into int
    for i in use.columns[range(69, 73)] :
      use[i] = use[i].astype('str').apply(lambda x:x[:-5]).astype('int32')
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(transform date type) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    # transform bool type into int
    for i in use.columns :
      if(use[i].dtypes == 'bool') :
        use[i] = use[i].apply(int)
    
    te = time.time()
    print('Inner -', str(tidx) + ' (generate feature) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    
    return use


def find_index(data_col, val):
    val_list = []
    
    val_list.append(val)
    val_list.append("end")

    index = data_col.isin(val_list).idxmax()
    
    return index


# =============================================================================
# Start Generate
# =============================================================================

import time
t1 = time.time()

to = 1
idx = 1
bc = True
row = 30000000
end = 349655789
Start = []
End = []

while(bc) :
    tm = 1
    
    t2 = time.time()

    if(end+1-idx <= row) :
      bc = False
      row = end+1-idx
      t2 = time.time()
      print('Stage -', str(to) + ' of ' + '(LASTTT) -> time elapsed: ' + str(round(t2-t1, 2)) + ' seconds' \
            + '\n\nStart : ' + str(idx) + ' End : ' + str(idx+row-1) + '\n')
      tm+=1

    print('\nStage -', str(to) + ' of ' + str(tm) + '(start) -> time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')
    tm+=1
    
    # from dask import dataframe as dd
    transactions = pd.read_csv(
        dirThis + 'transactions.csv', 
        names=['id', 'chain', 'dept', 'category', 'company', 'brand', 'date', \
               'productsize', 'productmeasure', 'purchasequantity', 'purchaseamount'],
        dtype = tranDtype,
        nrows = row,
        parse_dates=['date'],
        infer_datetime_format=True,
        skiprows = idx,
        # engine = "pyarrow",
        # engine = "python-fwf",
        engine = "c"
        # blocksize=64000000 # = 64 Mb chunks
    )
    
    if(bc) :
        idx1 = find_index(transactions['id'], int(transactions['id'].loc[len(transactions['id']) - 1]))
        
        del transactions
        gc.collect()
        
        t2 = time.time()
        print('Stage -', str(to) + ' of ' + str(tm) + '(read data) -> time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')
        tm+=1
        
        transactions = pd.read_csv(
            dirThis + 'transactions.csv', 
            names=['id', 'chain', 'dept', 'category', 'company', 'brand', 'date', \
                    'productsize', 'productmeasure', 'purchasequantity', 'purchaseamount'],
            dtype = tranDtype,
            nrows = idx1,
            parse_dates=['date'],
            infer_datetime_format=True,
            skiprows = idx,
            # engine = "pyarrow",
            # engine = "python-fwf",
            engine = "c"
            # blocksize=64000000 # = 64 Mb chunks
        )
    
    t2 = time.time()
    print('Stage -', str(to) + ' of ' + str(tm) + '(reRead data) -> time elapsed: ' + str(round(t2-t1, 2)) + ' seconds' \
          + '\n\nStart : ' + str(idx) + ' End : ' + str(idx+idx1-1) + '\n')
    tm+=1

    Start.append(idx)
    End.append(idx+idx1-1)
    
    idx += idx1
      
    use = generateFeature(offers, transactions, trainHistory, testHistory, to)

    del transactions
    gc.collect()
    
    t2 = time.time()
    print('Stage -', str(to) + ' of ' + str(tm) + '(generate) -> time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')
    tm+=1
    
    use.to_csv(dirThis + '/split/split' + str(to) + '.csv')
    
    
    del use
    
    t2 = time.time()
    print('Stage -', str(to) + ' of ' + str(tm) + '(ouput) -> time elapsed: ' + str(round(t2-t1, 2)) + ' seconds\n')
    gc.collect()
    tm+=1
    to+=1

print("Finish!!!!!")

In [5]:
import pandas as pd
import time

tranDtype = {
    'id': 'uint64',
    'chain': 'uint16',
    'dept': 'uint8',
    'category': 'uint16',
    'company': 'uint64',
    'brand': 'uint32',
    'date' : 'object',
    'productsize': 'float32',
    'productmeasure': 'category',
    'purchasequantity': 'int64',
    'purchaseamount': 'float32'
    }

dirThis = '/content/drive/MyDrive/1經濟學/專題/'

# offers = pd.read_csv(dirThis + 'offers.csv')

In [11]:
# !pip show pandas
# !pip uninstall pandas
# !pip install pandas==0.25.3

Found existing installation: pandas 2.0.0
Uninstalling pandas-2.0.0:
  Would remove:
    /usr/local/lib/python3.9/dist-packages/pandas-2.0.0.dist-info/*
    /usr/local/lib/python3.9/dist-packages/pandas/*
Proceed (Y/n)? Y
  Successfully uninstalled pandas-2.0.0


In [2]:
pip install dask toolz cloudpickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import dask.dataframe as dd

# df = dd.read_csv("large_dataset.csv")


In [12]:
# import pandas as pd

import dask.dataframe as dd
import time

dirThis = '/content/drive/MyDrive/1經濟學/專題/'
idx = 1
row = 15000000

t1 = time.time()
transactions = dd.read_csv(dirThis + 'newdata.csv', dtype = tranDtype)
t2 = time.time()

print('time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')

time elapsed: 0.11 seconds


In [13]:
transactions.info(memory_usage='deep')
transactions.shape

<class 'dask.dataframe.core.DataFrame'>
Columns: 11 entries, id to purchaseamount
dtypes: category(1), object(1), float32(2), int64(1), uint16(2), uint32(1), uint64(2), uint8(1)
memory usage: 731.9 MB


(Delayed('int-79dcb5a1-334a-4857-96dd-d361c5723e6a'), 11)

In [18]:
pd.DataFrame(transactions)

,0,1,2,3,4,5,6,7,8,9,10
0,86246,205,99,9909,104538848,15343,2012-03-02,16.0,OZ,1,2.49
1,86246,205,58,5824,108674585,55172,2012-03-02,16.0,OZ,1,3.29
2,86246,205,72,7205,103500030,3830,2012-03-06,4.6,OZ,1,3.99
3,86246,205,55,5558,104154848,5603,2012-03-07,5.8,OZ,1,1.25
4,86246,205,58,5824,1076401474,304,2012-03-14,12.0,OZ,1,4.99
...,...,...,...,...,...,...,...,...,...,...,...
15349951,4847628950,166,58,5824,1076401474,304,2013-07-23,12.0,OZ,1,5.99
15349952,4847787712,46,21,2119,1088692686,11473,2013-07-21,128.0,OZ,1,1.99
15349953,4847787712,46,35,3509,103320030,875,2013-07-21,75.0,OZ,2,6.98
15349954,4853598737,46,55,5558,104154848,5603,2013-07-20,5.8,OZ,3,3.0


In [ ]:
pd.io.parquet.PyArrowImpl()

idx = 1 
t1 = time.time()
# from dask import dataframe as dd
transactions = pd.read_csv(
    dirThis + 'transactions.csv', 
    names=['id', 'chain', 'dept', 'category', 'company', 'brand', 'date', \
           'productsize', 'productmeasure', 'purchasequantity', 'purchaseamount'],
    dtype = tranDtype,
    nrows = 100000,
    parse_dates=['date'],
    infer_datetime_format=True,
    header = idx
    # blocksize=64000000 # = 64 Mb chunks
)
# transactions = read_single_csv(dirThis + 'transactions.csv', idx)
t2 = time.time()

print('time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')

# pip install modin[ray]

import modin.pandas as pd
import ray
ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})

import pandas as pd
idx = 1000000
t1 = time.time()
# from dask import dataframe as dd
transactions = pd.read_csv(
    dirThis + 'transactions.csv', 
    names=['id', 'chain', 'dept', 'category', 'company', 'brand', 'date', \
           'productsize', 'productmeasure', 'purchasequantity', 'purchaseamount'],
    dtype = tranDtype,
    nrows = 1000000,
    parse_dates=['date'],
    infer_datetime_format=True,
    skiprows = idx,
    # engine = "pyarrow",
    # engine = "python-fwf",
    engine = "pyarrow"
    # blocksize=64000000 # = 64 Mb chunks
)
# transactions = read_single_csv(dirThis + 'transactions.csv', idx)
t2 = time.time()

print('time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')

pd.io.parquet.get_engine('auto')
import pyarrow.parquet